In [62]:
!pip install sastrawi

^C


In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)

from keras.layers import Dense, LeakyReLU, SimpleRNN, LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras import models
from keras import layers

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime as dt
import string
import re

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

import csv
import requests
from io import StringIO

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jayaw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jayaw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jayaw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Memuat Dataset

In [111]:
app_reviews_df = pd.read_csv('dataset.csv')

In [112]:
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

In [113]:
app_reviews_df.head()

,Review
0,"Rusak scan,Qris sulit terbaca kadang tidak bisa ngebaca..!!! Sulit bener untuk bertransaksi selalu ada hambatan...!! Sudah puluhan kali transfer selalu terkendala, anehnya setelah memberi ulasan di playstore langsung bisa transaksi..!!!! Transaksi selalu gagal... capek... ngulang ngulang...muter muter...!!!"
1,"Apk udah bagus tapi masih berasa berat , kalau bisa latar gambar yang ada di dalam apk jangan di buat gambar bergerak . Seperti nya karna gambar gerak itu apk dana jadi berat . Pencet fitur aja gk langsung masuk tapi ada stuck nya . Sama 1 lagi bebas biaya admin jangan di hapus , sebab itu buat pengguna yang kurang mampu jadi nyaman dan mudah . Trus adakan bebas biaya admin jangan di hapus."
2,"Aplikasinya sangat bagus diawal2 nya terutama transaksinya sangat membantu, tapi untuk update nya sangat lemot sekali dan saat sekarang juga kalau buka aplikasinya sangat lambat sekali alias lemot..jadi kalau mau transaksi saat ini sangat kesel harus nya segera dong di benahi..makasih"
3,Animasi promo yang kalian buat membuat aplikasi nya menjadi patah patah saat digunakan terlebih ketika meng-scroll app nya. Bisakah untuk setidaknya mengurangi animasinya atau membuat opsi di pengaturan terkait hal tersebut
4,"aplikasi makin kesini makin lemot saja... padahal koneksi bagus, sinyal manteng, kenapa pas awal buka dana jadi lemot, apalagi pas mau transaksi makin lemot saja... tolong untuk pihak pengembang untuk di perbaiki permasalahan lemot ini... coba di uji coba pakai hp jadul yang ram kecil, karena gak semua pengguna aplikasi dana mempunyai hp yang speknya bagus. terimakasih."


# Menghapus Data Kosong dan Data Duplikat

In [114]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126000 entries, 0 to 125999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  126000 non-null  object
dtypes: object(1)
memory usage: 984.5+ KB


In [115]:
clean_df = app_reviews_df.dropna()

In [116]:
clean_df.drop_duplicates(inplace=True)

In [117]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124209 entries, 0 to 125999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  124209 non-null  object
dtypes: object(1)
memory usage: 1.9+ MB


In [118]:
jumlah_ulasan_bersih, jumlah_kolom_bersih = clean_df.shape

#Preprocessing Text

In [119]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')

    return text

In [120]:
def casefoldingText(text):
    text = text.lower()
    return text

In [121]:
def tokenizingText(text):
    text = word_tokenize(text)
    return text

In [122]:
def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku','di','ga','ya','gaa','loh','kah','woi','woii','woy'])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

In [123]:
def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()

    stemmed_words = [stemmer.stem(word) for word in words]

    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

In [124]:
def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [125]:
slangwords = {'@': 'di', 'abis': 'habis', 'wtb': 'beli', 'masi': 'masih', 'wts': 'jual', 'wtt': 'tukar', 'bgt': 'banget', 'maks': 'maksimal'}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [126]:
clean_df['text_clean'] = clean_df['Review'].apply(cleaningText)
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)
clean_df['text_stopwords'] = clean_df['text_tokenizingText'].apply(filteringText)
clean_df['final_text'] = clean_df['text_stopwords'].apply(toSentence)

In [127]:
clean_df.head()

,Review,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopwords,final_text
0,"Rusak scan,Qris sulit terbaca kadang tidak bisa ngebaca..!!! Sulit bener untuk bertransaksi selalu ada hambatan...!! Sudah puluhan kali transfer selalu terkendala, anehnya setelah memberi ulasan di playstore langsung bisa transaksi..!!!! Transaksi selalu gagal... capek... ngulang ngulang...muter muter...!!!",Rusak scanQris sulit terbaca kadang tidak bisa ngebaca Sulit bener untuk bertransaksi selalu ada hambatan Sudah puluhan kali transfer selalu terkendala anehnya setelah memberi ulasan di playstore langsung bisa transaksi Transaksi selalu gagal capek ngulang ngulangmuter muter,rusak scanqris sulit terbaca kadang tidak bisa ngebaca sulit bener untuk bertransaksi selalu ada hambatan sudah puluhan kali transfer selalu terkendala anehnya setelah memberi ulasan di playstore langsung bisa transaksi transaksi selalu gagal capek ngulang ngulangmuter muter,rusak scanqris sulit terbaca kadang tidak bisa ngebaca sulit bener untuk bertransaksi selalu ada hambatan sudah puluhan kali transfer selalu terkendala anehnya setelah memberi ulasan di playstore langsung bisa transaksi transaksi selalu gagal capek ngulang ngulangmuter muter,"[rusak, scanqris, sulit, terbaca, kadang, tidak, bisa, ngebaca, sulit, bener, untuk, bertransaksi, selalu, ada, hambatan, sudah, puluhan, kali, transfer, selalu, terkendala, anehnya, setelah, memberi, ulasan, di, playstore, langsung, bisa, transaksi, transaksi, selalu, gagal, capek, ngulang, ngulangmuter, muter]","[rusak, scanqris, sulit, terbaca, kadang, ngebaca, sulit, bener, bertransaksi, hambatan, puluhan, kali, transfer, terkendala, anehnya, ulasan, playstore, langsung, transaksi, transaksi, gagal, capek, ngulang, ngulangmuter, muter]",rusak scanqris sulit terbaca kadang ngebaca sulit bener bertransaksi hambatan puluhan kali transfer terkendala anehnya ulasan playstore langsung transaksi transaksi gagal capek ngulang ngulangmuter muter
1,"Apk udah bagus tapi masih berasa berat , kalau bisa latar gambar yang ada di dalam apk jangan di buat gambar bergerak . Seperti nya karna gambar gerak itu apk dana jadi berat . Pencet fitur aja gk langsung masuk tapi ada stuck nya . Sama 1 lagi bebas biaya admin jangan di hapus , sebab itu buat pengguna yang kurang mampu jadi nyaman dan mudah . Trus adakan bebas biaya admin jangan di hapus.",Apk udah bagus tapi masih berasa berat kalau bisa latar gambar yang ada di dalam apk jangan di buat gambar bergerak Seperti nya karna gambar gerak itu apk dana jadi berat Pencet fitur aja gk langsung masuk tapi ada stuck nya Sama lagi bebas biaya admin jangan di hapus sebab itu buat pengguna yang kurang mampu jadi nyaman dan mudah Trus adakan bebas biaya admin jangan di hapus,apk udah bagus tapi masih berasa berat kalau bisa latar gambar yang ada di dalam apk jangan di buat gambar bergerak seperti nya karna gambar gerak itu apk dana jadi berat pencet fitur aja gk langsung masuk tapi ada stuck nya sama lagi bebas biaya admin jangan di hapus sebab itu buat pengguna yang kurang mampu jadi nyaman dan mudah trus adakan bebas biaya admin jangan di hapus,apk udah bagus tapi masih berasa berat kalau bisa latar gambar yang ada di dalam apk jangan di buat gambar bergerak seperti nya karna gambar gerak itu apk dana jadi berat pencet fitur aja gk langsung masuk tapi ada stuck nya sama lagi bebas biaya admin jangan di hapus sebab itu buat pengguna yang kurang mampu jadi nyaman dan mudah trus adakan bebas biaya admin jangan di hapus,"[apk, udah, bagus, tapi, masih, berasa, berat, kalau, bisa, latar, gambar, yang, ada, di, dalam, apk, jangan, di, buat, gambar, bergerak, seperti, nya, karna, gambar, gerak, itu, apk, dana, jadi, berat, pencet, fitur, aja, gk, langsung, masuk, tapi, ada, stuck, nya, sama, lagi, bebas, biaya, admin, jangan, di, hapus, sebab, itu, buat, pengguna, yang, kurang, mampu, jadi, nyaman, dan, mudah, trus, adakan, bebas, biaya, admin, jangan, di, hapus]","[apk, udah, bagus, 

# Memberi Label pada Data

In [128]:
lexicon_positive = dict()
lexicon_negative = dict()

In [129]:
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

In [130]:
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_positive[row[0]] = int(row[1])

else:
    print('Gagal')

In [131]:
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

In [132]:
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_negative[row[0]] = int(row[1])

else:
    print('Gagal')

In [133]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0

    for word in text:
        if(word in lexicon_positive):
            score = score + lexicon_positive[word]

    for word in text:
        if(word in lexicon_negative):
            score = score + lexicon_negative[word]

    polarity=''

    if(score>0):
        polarity = 'positive'
    elif(score<0):
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity

In [134]:
results = clean_df['text_stopwords'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    63540
negative    52433
neutral      8236
Name: count, dtype: int64


# Data Splitting dan Ektraksi Fitur

In [135]:
x = clean_df['final_text']
y = clean_df['polarity']

In [136]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [137]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y_encoded, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [138]:
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)

In [139]:
x_train_tfidf = tfidf.fit_transform(x_train)
x_val_tfidf = tfidf.transform(x_val)
x_test_tfidf = tfidf.transform(x_test)

In [140]:
features_df = pd.DataFrame(x_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [141]:
features_df

,admin,aja,aktif,akun,alasan,aman,aneh,apapun,apk,aplikasi,...,ulang,update,upgrade,verifikasi,versi,via,voucher,wa,wajah,yg
0,0.0,0.000000,0.0,0.0,0.0,0.335198,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.222624
1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.215716,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.318042,...,0.338364,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.207114,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99362,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.233016,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
99363,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.262073,...,0.000000,0.394254,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
99364,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.159598,...,0.000000,0.000000,0.0,0.0,0.0,0.173676,0.0,0.0,0.0,0.000000
99365,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.165260,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


# Pembuatan Model

In [142]:
OUTPUT_SIZE = len(y.unique())
OUTPUT_SIZE

3

In [143]:
model = Sequential([
    LSTM(8, return_sequences=True, input_shape=(1, x_train_tfidf.shape[1])),
    LSTM(16, return_sequences=True),
    LSTM(32),
    Dense(16),
    LeakyReLU(),
    Dense(OUTPUT_SIZE, activation='softmax')
])

In [144]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

In [145]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 1, 8)              6688      
                                                                 
 lstm_10 (LSTM)              (None, 1, 16)             1600      
                                                                 
 lstm_11 (LSTM)              (None, 32)                6272      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 3)                 51        
                                                                 
Total params: 15139 (59.14 KB)
Trainable params: 15139

# Pelatihan Model

In [146]:
x_train_reshaped = np.reshape(x_train_tfidf.toarray(), (x_train_tfidf.shape[0], 1, x_train_tfidf.shape[1]))
x_val_reshaped = np.reshape(x_val_tfidf.toarray(), (x_val_tfidf.shape[0], 1, x_val_tfidf.shape[1]))

model.fit(x_train_reshaped, y_train, epochs=10, validation_data=(x_val_reshaped, y_val))

Epoch 1/10
3106/3106 [==============================] - 57s 12ms/step - loss: 0.5873 - accuracy: 0.7521 - val_loss: 0.4889 - val_accuracy: 0.8151
Epoch 2/10
3106/3106 [==============================] - 42s 14ms/step - loss: 0.4858 - accuracy: 0.8176 - val_loss: 0.4702 - val_accuracy: 0.8226
Epoch 3/10
3106/3106 [==============================] - 43s 14ms/step - loss: 0.4765 - accuracy: 0.8221 - val_loss: 0.4665 - val_accuracy: 0.8261
Epoch 4/10
3106/3106 [==============================] - 42s 14ms/step - loss: 0.4725 - accuracy: 0.8238 - val_loss: 0.4629 - val_accuracy: 0.8259
Epoch 5/10
3106/3106 [==============================] - 42s 14ms/step - loss: 0.4699 - accuracy: 0.8242 - val_loss: 0.4630 - val_accuracy: 0.8258
Epoch 6/10
3106/3106 [==============================] - 43s 14ms/step - loss: 0.4676 - accuracy: 0.8252 - val_loss: 0.4580 - val_accuracy: 0.8289
Epoch 7/10
3106/3106 [==============================] - 42s 14ms/step - loss: 0.4657 - accuracy: 0.8253 - val_loss: 0.4613 -

# Prediksi Model

In [147]:
x_test_reshaped = np.reshape(x_test_tfidf.toarray(), (x_test_tfidf.shape[0], 1, x_test_tfidf.shape[1]))

In [148]:
y_pred = model.predict(x_test_reshaped)

389/389 [==============================] - 4s 4ms/step


# Evaluasi Model

In [149]:
y_pred_classes = np.argmax(y_pred, axis=1)

In [150]:
accuracy = round(accuracy_score(y_pred_classes, y_test), 4)

In [151]:
print('Akurasi Model LSTM :', accuracy)

Akurasi Model LSTM : 0.8323


In [152]:
print(classification_report(y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84      5323
           1       0.52      0.24      0.33       758
           2       0.84      0.89      0.87      6340

    accuracy                           0.83     12421
   macro avg       0.73      0.66      0.68     12421
weighted avg       0.82      0.83      0.82     12421



In [153]:
confusion_matrix(y_test, y_pred_classes)

array([[4488,   68,  767],
       [ 251,  181,  326],
       [ 573,   98, 5669]], dtype=int64)

# Contoh Input

In [158]:
test_sample = x_test.sample(n=5)
test_sample

4973                                                                                                           aplikasi otak akun saldo rp login gara ² kode otp nomor mati verifekasi pake wajah aneh nian aplikasi tolong bantu dana hubungi email
12528     apapun tu tf temen dgn nominal tau taunya uang hilang riwayat pengirimannya saldo berkurang aplikasinya aneh sorry say kecewa banget aplikasinya aplikasinya amanuntuk pengiriman gangguan gangguan gangguan sinyal bagus edit ulasan uang
123681                                                                                                                                                                 memuaskan harga transaksinya memuaskan hati tiada kendala lambat bertransaksi
25666                                           topup game dana bagus topup toko yg pembayaran riwayat transaksinya susah membayar transaksi mencatat solusi youtube google penyelesaiannya solusinya kecewa tolong perbaiki aplikasinya update biar
97188               

In [159]:
test_sample_tfidf = tfidf.transform(test_sample)
test_sample_reshaped = np.reshape(test_sample_tfidf.toarray(), (test_sample_tfidf.shape[0], 1, test_sample_tfidf.shape[1]))

In [160]:
predicted_values = model.predict(test_sample_reshaped)
predicted_classes = np.argmax(predicted_values, axis=1)
sentiment = encoder.inverse_transform(predicted_classes)

1/1 [==============================] - 0s 147ms/step


In [161]:
pd.set_option('display.max_colwidth', None)

pd.DataFrame({'text': test_sample, 'sentiment': sentiment})

,text,sentiment
4973,aplikasi otak akun saldo rp login gara ² kode otp nomor mati verifekasi pake wajah aneh nian aplikasi tolong bantu dana hubungi email,negative
12528,apapun tu tf temen dgn nominal tau taunya uang hilang riwayat pengirimannya saldo berkurang aplikasinya aneh sorry say kecewa banget aplikasinya aplikasinya amanuntuk pengiriman gangguan gangguan gangguan sinyal bagus edit ulasan uang,negative
123681,memuaskan harga transaksinya memuaskan hati tiada kendala lambat bertransaksi,positive
25666,topup game dana bagus topup toko yg pembayaran riwayat transaksinya susah membayar transaksi mencatat solusi youtube google penyelesaiannya solusinya kecewa tolong perbaiki aplikasinya update biar,positive
97188,mohon perbaiki lahmasa gabisa transaksi gabisa ngirim masuk aplikasi udah kek gtuu trs,negative
